### Data Preprocessing

In [14]:
import pandas as pd
survey = pd.read_csv("cleaned_survey_results_2019.csv")
survey.head()

Have you studied data visualization in school (or other formal environment) or did you learn how to do it on your own?  \
0                 Equal Parts School and Self-Taught                                                                       
1                                 Mostly Self-Taught                                                                       
2                 Equal Parts School and Self-Taught                                                                       
3                                 Mostly Self-Taught                                                                       
4                                 Mostly Self-Taught                                                                       

  What methods do you think best teach data visualization?  \
0  Workshops, Collaboration with more skilled pra...         
1  Workshops, Video Tutorials, Collaboration with...         
2  Examples, Video Tutorials, Working through a p...         
3                Examples, Working through a project         
4                Examples, Working through a project         

  What area is your next priority for improving your data visualization skills?  \
0  Improving my skills with an existing technical...                              
1           Learning a new technical tool or library                              
2                           Improving my data skills                              
3                           Improving my data skills                              
4                           Improving my data skills                              

  Are you a freelancer/consultant?  \
0                               No   
1                               No   
2                               No   
3                              Yes   
4                              Yes   

      What focus is data visualization in your work?  \
0  Data visualization is an important secondary p...   
1          Data visualization is the focus of my job   
2  Data visualization is one of several other thi...   
3  Data visualization is an important secondary p...   
4  Data visualization is an important secondary p...   

  For the majority of your projects: were you hired to do data visualization only or is data visualization a part of your job/project description?  \
0          Data visualization is only part of my job                                                                                                 
1          Data visualization is only part of my job                                                                                                 
2          Data visualization is only part of my job                                                                                                 
3          Data visualization is only part of my job                                                                                                 
4          Data visualization is only part of my job                                                                                                 

  Do you do more or less data visualization than you expected for this role?  \
0          About as much as I expected for this role                           
1                                               Less                           
2                                               Less                           
3          About as much as I expected for this role                           
4          About as much as I expected for this role                           

  Which one of these is the closest to describing your role?  \
0                                           Academic           
1            Leadership (Manager, Director, VP, etc)           
2            Leadership (Manager, Director, VP, etc)           
3            Leadership (Manager, Director, VP, etc)           
4            Leadership (Manager, Director, VP, etc)           

  What is your yearly pay? What is your hourly rate?

In [15]:
# select columns of interest
used_columns = ['Have you studied data visualization in school (or other formal environment) or did you learn how to do it on your own?', 
                            'Which one of these is the closest to describing your role?', 
                            'What is your yearly pay?', 
                            'What technologies do you use to visualize data? Select all that apply.',
                            'Which of these charts have you used in production in the last 6 months? Select all that apply.',
                            'The organization you work for is in which of the following areas?',
                            'If you went to college, what did you major in during your undergraduate work?',
                            'What\'s your gender identity?',
                            'What country do you live in?']

survey = survey[used_columns]

# drop rows with missing value
survey = survey.dropna(axis=0)

# rename columns as their names are too long
survey = survey.rename(columns={"What technologies do you use to visualize data? Select all that apply.": "Visualisation_tool", 
                                "Which of these charts have you used in production in the last 6 months? Select all that apply.": "Charts",
                               "Which one of these is the closest to describing your role?": "Role", 
                               "Have you studied data visualization in school (or other formal environment) or did you learn how to do it on your own?": "Learning_method",
                               "The organization you work for is in which of the following areas?": "Organization_area",
                               "If you went to college, what did you major in during your undergraduate work?": "Undergraduate_major",
                               "What's your gender identity?": "Gender",
                               "What country do you live in?": "Country",
                               "What is your yearly pay?": "Yearly_pay"})

survey.columns

Index(['Learning_method', 'Role', 'Yearly_pay', 'Visualisation_tool', 'Charts',
       'Organization_area', 'Undergraduate_major', 'Gender', 'Country'],
      dtype='object')

In [16]:
# simplify value labeling
survey.loc[survey['Role'] == "Leadership (Manager, Director, VP, etc)", 'Role'] = "Leadership"
survey.loc[survey['Role'] == "None of these describe my role", 'Role'] = "Other"
survey.loc[survey['Yearly_pay'] == "Less than $20,000", 'Yearly_pay'] = "< $20k"

# remove white space in the cell
survey['Visualisation_tool']=survey['Visualisation_tool'].str.replace('\s','')
survey['Charts']=survey['Charts'].str.replace('\s','')

<ipython-input-16-13e5f4c7a19d>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  survey['Visualisation_tool']=survey['Visualisation_tool'].str.replace('\s','')
<ipython-input-16-13e5f4c7a19d>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  survey['Charts']=survey['Charts'].str.replace('\s','')


In [7]:
# get a list of visualisation tools with at least 10 frequency
rows_technology = []
for row in survey['Visualisation_tool']:
    rows_technology+=row.split(",")
technology_counts = pd.Series(rows_technology).value_counts()
print(technology_counts[technology_counts >= 100])
tech_lists = technology_counts[technology_counts >= 100].keys()
tech_lists

Excel          593
Tableau        486
R              410
ggplot2        360
D3             360
Python         355
Pen&Paper      323
Illustrator    276
PowerBI        187
Plotly         170
Mapbox         166
QGIS           140
Leaflet        129
ArcGIS         115
Matplotlib     113
React          109
dtype: int64


Index(['Excel', 'Tableau', 'R', 'ggplot2', 'D3', 'Python', 'Pen&Paper',
       'Illustrator', 'PowerBI', 'Plotly', 'Mapbox', 'QGIS', 'Leaflet',
       'ArcGIS', 'Matplotlib', 'React'],
      dtype='object')

In [17]:
# one-hot encoding tool used
for name in tech_lists:
    survey["tool_" + name] = survey['Visualisation_tool'].apply(lambda x:'yes' if name in x else 'no')

In [18]:
# get a list of charts used with at least 10 frequency
rows_chart = []
for row in survey['Charts']:
    rows_chart+=row.split(",")
chart_counts = pd.Series(rows_chart).value_counts()
print(chart_counts[chart_counts >= 100])
chart_lists = chart_counts[chart_counts >= 100].keys()
chart_lists

BarChart                  981
LineChart                 969
Scatterplot               767
PieChart                  449
Hexbin/Heatmap            424
Infographics              373
Treemap                   371
FlowChart)                359
FlowDiagram(Sankey        359
DAGRE                     359
ChoroplethMap             333
NetworkDiagram            256
PictorialVisualization    174
Force-DirectedGraph       130
RasterMap                 118
dtype: int64


Index(['BarChart', 'LineChart', 'Scatterplot', 'PieChart', 'Hexbin/Heatmap',
       'Infographics', 'Treemap', 'FlowChart)', 'FlowDiagram(Sankey', 'DAGRE',
       'ChoroplethMap', 'NetworkDiagram', 'PictorialVisualization',
       'Force-DirectedGraph', 'RasterMap'],
      dtype='object')

In [19]:
# one-hot encoding charts used
for name in chart_lists:
    survey["chart_" + name] = survey['Charts'].apply(lambda x: 'yes' if name in x else 'no')

In [12]:
survey.columns

Index(['Learning_method', 'Role', 'Yearly_pay', 'Visualisation_tool', 'Charts',
       'Organization_area', 'Undergraduate_major', 'Gender', 'Country',
       'tool_Excel', 'tool_Tableau', 'tool_R', 'tool_ggplot2', 'tool_D3',
       'tool_Python', 'tool_Pen&Paper', 'tool_Illustrator', 'tool_PowerBI',
       'tool_Plotly', 'tool_Mapbox', 'tool_QGIS', 'tool_Leaflet',
       'tool_ArcGIS', 'tool_Matplotlib', 'tool_React', 'chart_BarChart',
       'chart_LineChart', 'chart_Scatterplot', 'chart_PieChart',
       'chart_Hexbin/Heatmap', 'chart_Infographics', 'chart_Treemap',
       'chart_FlowChart)', 'chart_FlowDiagram(Sankey', 'chart_DAGRE',
       'chart_ChoroplethMap', 'chart_NetworkDiagram',
       'chart_PictorialVisualization', 'chart_Force-DirectedGraph',
       'chart_RasterMap'],
      dtype='object')

In [13]:
survey.to_csv("processed_survey.csv")

In [ ]:
### 
```{r data preprocessing}
# import data
survey <- read_csv("data_visualization_survey-master/data/processed_survey.csv")
survey <- survey[,2:ncol(survey)]
survey
col_list <- colnames(survey)
col_list
```


```{r}
# tool and role
selected_tool <- c(col_list[2:3], col_list[10:25])
tool <- survey %>% group_by(survey[selected_tool]) %>%
summarise(freq = n()) #%>% filter(freq > 1)
#gather the data.frame into long form
tool <- gather_set_data(tool, 1:18)
#plot parallel set
ggplot(tool, aes(x, id = id, split = y, value = freq)) + 
geom_parallel_sets(aes(fill = `Role`), alpha = 0.3, axis.width = 0.2) +
geom_parallel_sets_axes(axis.width = 0.2) +
geom_parallel_sets_labels(colour = 'orangered1', angle=360, size = 4) +
theme(axis.text.x = element_text(angle = 90))
```
```{r}
selected_chart <- c(col_list[2:3], col_list[26:40])
chart <- survey %>% group_by(survey[selected_chart]) %>%
summarise(freq = n()) #%>% filter(freq > 1) %>% arrange(desc(freq))

#gather the data.frame into long form
chart <- gather_set_data(chart, 1:17)
#plot parallel set
ggplot(chart, aes(x, id = id, split = y, value = freq)) + 
geom_parallel_sets(aes(fill = `Role`), alpha = 0.3, axis.width = 0.2) +
geom_parallel_sets_axes(axis.width = 0.2) +
geom_parallel_sets_labels(colour = 'orangered1', angle=360, size = 4) +
theme(axis.text.x = element_text(angle = 90))
```
```{r}
ggparcoord(data = survey, 
           columns = c(2:3,6:9), 
           groupColumn = 11,
           scale = "uniminmax",
           showPoints = TRUE,
           boxplot = TRUE, 
           title = "Plot of data visualisation survey") +
  theme(axis.text.x = element_text(angle = 90))
```
```{r}
ggparcoord(data = survey, 
           columns = c(10:25), 
           groupColumn = 3,
           scale = "uniminmax",
           showPoints = TRUE,
           boxplot = TRUE, 
           title = "Plot of data visualisation survey") +
  theme(axis.text.x = element_text(angle = 90))
```
```{r}
ggparcoord(data = survey, 
           columns = c(1:2, 6:9), 
           groupColumn = 3,
           scale = "uniminmax",
           showPoints = TRUE,
           boxplot = TRUE, 
           title = "Parallel Coord. Plot of data visualisation survey") +
  theme(axis.text.x = element_text(angle = 90))
